In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from os import listdir

import wfdb #PyPi package for loading ecg and annotations

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import wrangle

In [2]:
X_train, X_validate, X_test, Y_train, Y_validate, Y_test, sym_train, sym_validate, sym_test, p_signal, atr_sym, atr_sample, ab_index = wrangle.wrangle()

Define path and patient numbers...
Annotations loaded...
Loading ecg files...
Creating dataset...
(This usually takes a while...)
Dataframes created...
Splitting train, validate, test...
This also takes a while...
All done.


## Make random forest models varying hyperparameters and evaluate performance

In [3]:
# Parameters

min_samples_leaf=3
n_estimators=100
max_depth=3

In [4]:
# Make the object
rf1 = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=min_samples_leaf,
                            n_estimators=n_estimators,
                            max_depth=max_depth, 
                            random_state=123)

In [5]:
Y_train.shape

(78551, 1)

In [6]:
Y_train.sample(20)

,0
66160,0.0
31685,0.0
8790,1.0
94326,0.0
94997,0.0
108746,0.0
21798,1.0
107560,0.0
18983,0.0
45674,0.0


In [7]:
# Fit the object
rf1.fit(X_train, Y_train)


/var/folders/9y/17ptyrds267724n2_7s7hx340000gn/T/ipykernel_37553/1936475531.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf1.fit(X_train, Y_train)


RandomForestClassifier(max_depth=3, min_samples_leaf=3, random_state=123)

In [9]:
#Make Predictions
y_pred_train_rf1 = rf1.predict(X_train)
y_pred_validate_rf1 = rf1.predict(X_validate)


In [11]:
#Make evaluation tables
train_eval = Y_train.copy()
#train_eval.rename(columns = {'0': 'actual'}, inplace=True)


In [13]:
train_eval['rf1_prediction'] = y_pred_train_rf1

validate_eval = Y_validate.copy()
#validate_eval.rename(columns = {'0': 'actual'}, inplace=True)
validate_eval['rf1_prediction'] = y_pred_validate_rf1

In [15]:
train_eval

,0,rf1_prediction
39859,0.0,0.0
35749,0.0,0.0
96921,0.0,0.0
63295,0.0,0.0
80479,0.0,0.0
...,...,...
4741,1.0,1.0
80756,0.0,0.0
105189,0.0,0.0
40237,0.0,0.0


In [16]:
train_eval.columns.values[0]='actual'
validate_eval.columns.values[0]='actual'

In [17]:
validate_eval.sample(20)

,actual,rf1_prediction
60390,1.0,0.0
82258,1.0,1.0
100084,1.0,1.0
19134,1.0,1.0
32215,0.0,0.0
1798,0.0,0.0
40927,0.0,0.0
57149,0.0,0.0
36649,1.0,0.0
49779,0.0,0.0


In [18]:
train_eval.sample(20)

,actual,rf1_prediction
66557,0.0,0.0
55380,0.0,0.0
63804,1.0,1.0
15954,1.0,1.0
14241,1.0,0.0
88512,0.0,0.0
103147,1.0,0.0
100554,1.0,0.0
81668,0.0,0.0
107356,0.0,0.0


In [19]:
print(X_train.shape)
print(Y_train.shape)

(78551, 2160)
(78551, 1)


In [20]:
Y_train.head()

,0
39859,0.0
35749,0.0
96921,0.0
63295,0.0
80479,0.0


No idea how the prediction got appended to the Y_train df. Can't find the code for it. Going to drop that so the dimensions line up for subsequent code

In [22]:
Y_train.head()

,0
39859,0.0
35749,0.0
96921,0.0
63295,0.0
80479,0.0


In [23]:
print('Accuracy of random forest classifier 1 on training set: {:.2f}'
     .format(rf1.score(X_train, Y_train)))

Accuracy of random forest classifier 1 on training set: 0.84


In [24]:
print(confusion_matrix(Y_train, y_pred_train_rf1))


[[53533   234]
 [12162 12622]]


In [25]:
print(classification_report(Y_train, y_pred_train_rf1))


              precision    recall  f1-score   support

         0.0       0.81      1.00      0.90     53767
         1.0       0.98      0.51      0.67     24784

    accuracy                           0.84     78551
   macro avg       0.90      0.75      0.78     78551
weighted avg       0.87      0.84      0.83     78551



In [26]:
print('Accuracy of random forest classifier 1 on validation set: {:.2f}'
     .format(rf1.score(X_validate, y_pred_validate_rf1)))

Accuracy of random forest classifier 1 on validation set: 1.00


In [27]:
print(classification_report(Y_validate, y_pred_validate_rf1))


              precision    recall  f1-score   support

         0.0       0.82      1.00      0.90     13538
         1.0       0.98      0.51      0.67      6100

    accuracy                           0.85     19638
   macro avg       0.90      0.75      0.79     19638
weighted avg       0.87      0.85      0.83     19638

